Syntax natürlicher Sprachen, WS 2023/24

# 07 - Aufgabenblatt (Lösung)


In [1]:
import nltk
from nltk import Tree
from nltk import DependencyGraph
from spacy import displacy
from itertools import chain

In [2]:
def _tree_labeled(self, i):
    node = self.get_by_address(i)
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word + '(' + rel + ')', [self._tree_labeled(dep) for dep in deps]) if deps else word + '(' + rel + ')'

def tree_labeled(self):
    node = self.root
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split(); line_list.pop(0); line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))
    return "\n".join([i for i in sent_list[0:]])

def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())
    deps = [x for x in deps if x]
    ex, word_list, arc_list = [], [], []
    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1; head = int(dep[2]); label = dep[3]
        if head>line:
            start = index; end = head-1; direction = "left"
        else:
            start = head-1; end = index; direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})
    ex.append({"words": word_list,"arcs": arc_list})
    return ex

def show_dep_trees(sent_nr,style=1):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)
    if style == 0 or style == 2:
        tree_labeled = dg.tree_labeled()
        tree_labeled.pretty_print(unicodelines=True)   
    if style == 1 or style == 2:
        ex = displacy_dep_input(sent)
        html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

## Aufgabe 1 - Analyse komplexer Satzkonstruktion

### a) Sehen Sie sich den folgenden englischen Satz an. Können Sie sich jeweils zwei unterschiedliche Interpretationen vorstellen?

- *Visiting relatives can be tiresome.*


In [ ]:
# Lösung:
# 1. Verwandte, die zu Besuch sind
# 2. Verwandte zu besuchen

### b) Erweitern Sie die gegebene Grammatik, so dass linguistisch korrekte Syntaxanalysen beider Interpretationen ausgegeben werden.

##### Verwenden Sie `VBG` für *Verb, gerund or present participle* (Penn-Treebank) und `VP-INF` für infinite Satzkonstruktionen.

In [13]:
sent = 'visiting relatives can be tiresome'

In [14]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> VBG N
    S -> S VP
    S -> VP
    VP -> VBG NP

    VP -> AUX VP | COP ADJ
    NP -> N

    VBG -> 'visiting'
    N -> 'relatives'
    AUX -> 'can'
    COP -> 'be'
    ADJ -> 'tiresome'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

                        S                  
          ┌─────────────┴───┐               
          │                 VP             
          │             ┌───┴───┐           
          NP            │       VP         
   ┌──────┴──────┐      │   ┌───┴─────┐     
  VBG            N     AUX COP       ADJ   
   │             │      │   │         │     
visiting     relatives can  be     tiresome

                        S                  
          ┌─────────────┴───┐               
          S                 │              
          │                 │               
          VP                VP             
   ┌──────┴──────┐      ┌───┴───┐           
   │             NP     │       VP         
   │             │      │   ┌───┴─────┐     
  VBG            N     AUX COP       ADJ   
   │             │      │   │         │     
visiting     relatives can  be     tiresome



### c) Geben Sie auch die entsprechenden Dependenzstrukturanalysen an.

In [5]:
# 'relatives' ist Subjekt des Hautsatzes, 'visiting' ist adnominaler Dependenten von 'relatives'
sent_nr = """
1 Visiting 2 acl
2 relatives 5 nsbuj
3 can 5 aux
4 be 5 cop
5 tiresome 0 ROOT
"""
show_dep_trees(sent_nr)

In [6]:
# 'visiting' ist Subjekt des Hauptsatzes (infiniter Subjektsatz), 'relatives' ist Objekt von 'visiting'
sent_nr = """
1 Visiting 5 csubj
2 relatives 1 obj
3 can 5 aux
4 be 5 cop
5 tiresome 0 ROOT
"""
show_dep_trees(sent_nr)

## Aufgabe 2 - Analyse komplexer Satzkonstruktion

#### Schreiben Sie eine CFG, so dass für den folgende Satz eine korrekte Syntaxanalyse ausgegeben wird. Geben Sie auch eine entsprechende Dependenzanalyse an.

- *Als ich ankam, habe ich gesehen, dass sie bereits dagewesen sein mussten.*


In [17]:
sent = 'als ich ankam   habe ich gesehen   dass sie bereits dagewesen sein mussten'

#### Konstituentenanalyse:

In [18]:
grammar = nltk.CFG.fromstring(
    """
    S -> SBAR AUX NP VP
    SBAR -> SCONJ S
    S -> NP VP
    VP -> V | V SBAR | ADV V AUX AUX

    NP ->  N

    N -> 'ich' | 'alles' | 'sie'
    V -> 'ankam' | 'gesehen' | 'dagewesen'
    AUX -> | 'habe'| 'sein' | 'mussten'
    SCONJ -> 'als' | 'dass'
    ADV -> 'bereits'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

                                 S                                             
       ┌──────────────┬────┬─────┴──────┐                                       
       │              │    │            VP                                     
       │              │    │     ┌──────┴────┐                                  
      SBAR            │    │     │          SBAR                               
  ┌────┴────┐         │    │     │      ┌────┴──────┐                           
  │         S         │    │     │      │           S                          
  │    ┌────┴────┐    │    │     │      │    ┌──────┴────────┐                  
  │    NP        VP   │    NP    │      │    NP              VP                
  │    │         │    │    │     │      │    │      ┌────────┼──────┬──────┐    
SCONJ  N         V   AUX   N     V    SCONJ  N     ADV       V     AUX    AUX  
  │    │         │    │    │     │      │    │      │        │      │      │    
 als  ich      ankam habe ich gese

#### Dependenzanalyse:

In [9]:
sent_nr = """
1 als 3 mark
2 ich 3 nsubj
3 ankam 6 advcl
4 habe 6 aux
5 ich 6 nsubj
6 gesehen 0 ROOT
7 dass 10 mark
8 sie 10 nsubj
9 bereits 10 advmod
10 dagewesen 6 ccomp
11 sein 10 aux
12 mussten 10 aux
"""

show_dep_trees(sent_nr)